# Finding the center

In [1]:
import numpy as np
import math
from numpy import array
from geopy.distance import geodesic
import gmplot

def weiszfeld(points):

    max_error = 0.0000000001

    x=np.array([point[0] for point in  points])
    y=np.array([point[1] for point in  points])


    ext_condition = True

    start_x = np.average(x)
    start_y = np.average(y)

    while ext_condition:

        sod = (((x - start_x)**2) + ((y - start_y)**2))**0.5

        new_x = sum(x/sod) / sum(1/sod)
        new_y = sum(y/sod) / sum(1/sod)

        ext_condition = (abs(new_x - start_x) > max_error) or (abs(new_y - start_y) > max_error)

        start_y = new_y
        start_x = new_x

        center_point = [start_x, start_y]

#     print(start_x, start_y)
    return calc_geodesic_dist(center_point, points)

#Using brute force method, can be optimised.
def calc_geodesic_dist(center_point, points):
    min_dist_from_center = 999999999999999999999999999999
    for point in points:
        if geodesic(point,center_point) < min_dist_from_center:
            min_dist_from_center = geodesic(point,center_point)
            center = point
    return center


if __name__=="__main__":
    # weiszfeld([(2,1), (12,2), (3,9), (13,11)]
    coordinates = [(49.276774,-123.12523), (49.841375,-119.487573), (49.500151,-117.283269), (49.23206,-123.032935), (51.08277, -114.154081)]
    center = weiszfeld(coordinates)
    print(center)

(49.841375, -119.487573)


# Request the Distance Matrix from Google API

In [129]:
import requests
import json
import urllib.parse
        
API_KEY = ""

# Used to parse into URL format
def urlparser(coordinates):
    n = len(coordinates)
    txt1 = [0] * n 
    for i in range(len(coordinates)):
        txt1[i] = ','.join(list(map(str, coordinates[i])))

    txt = '|'.join(txt1)
    return urllib.parse.quote(txt)

origins = urlparser(coordinates)
url = ('https://maps.googleapis.com/maps/api/distancematrix/json'
        + '?origins={}'
        + '&destinations={}'
        + '&key={}').format(origins, origins, API_KEY)

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

49.276774%2C-123.12523%7C49.841375%2C-119.487573%7C49.500151%2C-117.283269%7C49.23206%2C-123.032935%7C51.08277%2C-114.154081


In [ ]:
#Write to json
with open('distance.json', 'w') as f:
    json.dump(response.json(), f)

In [2]:
def tsp(data):
    # build a graph
    G = build_graph(data)
#   print("Graph: ", G)

    # build a minimum spanning tree
    MSTree = minimum_spanning_tree(G)
    # print("MSTree: ", MSTree)

    # find odd vertexes
    odd_vertexes = find_odd_vertexes(MSTree)
    # print("Odd vertexes in MSTree: ", odd_vertexes)

    # add minimum weight matching edges to MST
    minimum_weight_matching(MSTree, G, odd_vertexes)
    # print("Minimum weight matching: ", MSTree)

    # find an eulerian tour
    eulerian_tour = find_eulerian_tour(MSTree, G)

    # print("Eulerian tour: ", eulerian_tour)

    current = eulerian_tour[0]
    path = [current]
    visited = [False] * len(eulerian_tour)
    visited[eulerian_tour[0]] = True
    length = 0

    for v in eulerian_tour:
        if not visited[v]:
            path.append(v)
            visited[v] = True

            length += G[current][v]
            current = v

    length +=G[current][eulerian_tour[0]]
    path.append(eulerian_tour[0])

    print("Result path: ", path)
    print("Result length of the path: {} meter".format(length))

    return length, path

def build_graph(data):#O(n^2)
    # from json
    with open('distance.json') as  f:
        dist = json.load(f)
    
    graph = {}
    for point1 in range(len(data)):
        for point2 in range(len(data)):
            if point1 != point2:
                if point1 not in graph:
                    graph[point1] = {}

                graph[point1][point2] = dist['rows'][point1]['elements'][point2]['distance']['value']

    return graph


class UnionFind:
    def __init__(self):
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        return iter(self.parents)

    def union(self, *objects):
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r], r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest


def minimum_spanning_tree(G):
    tree = []
    subtrees = UnionFind()
    for W, u, v in sorted((G[u][v], u, v) for u in G for v in G[u]):
        if subtrees[u] != subtrees[v]:
            tree.append((u, v, W))
            subtrees.union(u, v)

    return tree


def find_odd_vertexes(MST):
    tmp_g = {}
    vertexes = []
    for edge in MST:
        if edge[0] not in tmp_g:
            tmp_g[edge[0]] = 0

        if edge[1] not in tmp_g:
            tmp_g[edge[1]] = 0

        tmp_g[edge[0]] += 1
        tmp_g[edge[1]] += 1

    for vertex in tmp_g:
        if tmp_g[vertex] % 2 == 1:
            vertexes.append(vertex)

    return vertexes


def minimum_weight_matching(MST, G, odd_vert):
    import random
    random.shuffle(odd_vert)

    while odd_vert:
        v = odd_vert.pop()
        length = float("inf")
        u = 1
        closest = 0
        for u in odd_vert:
            if v != u and G[v][u] < length:
                length = G[v][u]
                closest = u

        MST.append((v, closest, length))
        odd_vert.remove(closest)


def find_eulerian_tour(MatchedMSTree, G):
    # find neigbours
    neighbours = {}
    for edge in MatchedMSTree:
        if edge[0] not in neighbours:
            neighbours[edge[0]] = []

        if edge[1] not in neighbours:
            neighbours[edge[1]] = []

        neighbours[edge[0]].append(edge[1])
        neighbours[edge[1]].append(edge[0])

    # print("Neighbours: ", neighbours)

    # finds the hamiltonian circuit
    start_vertex = MatchedMSTree[0][0]
    EP = [neighbours[start_vertex][0]]

    while len(MatchedMSTree) > 0:
        for i, v in enumerate(EP):
            if len(neighbours[v]) > 0:
                break

        while len(neighbours[v]) > 0:
            w = neighbours[v][0]

            remove_edge_from_matchedMST(MatchedMSTree, v, w)

            del neighbours[v][(neighbours[v].index(w))]
            del neighbours[w][(neighbours[w].index(v))]

            i += 1
            EP.insert(i, w)

            v = w

    return EP


def remove_edge_from_matchedMST(MatchedMST, v1, v2):

    for i, item in enumerate(MatchedMST):
        if (item[0] == v2 and item[1] == v1) or (item[0] == v1 and item[1] == v2):
            del MatchedMST[i]

    return MatchedMST

if __name__=="__main__":
    print(coordinates)
    data_sorted = tsp(coordinates)

[(49.276774, -123.12523), (49.841375, -119.487573), (49.500151, -117.283269), (49.23206, -123.032935), (51.08277, -114.154081)]


NameError: name 'json' is not defined

# Plot the polygon

In [171]:
# create latitude and longitude with order
order = data_sorted[1]
lat = []
long = []
loc = []

for i in range(len(order)):
    lat.append(data[order[i]][0])
    long.append(data[order[i]][1])
    loc.append(data[order[i]])
    
print(lat)
print(long)
print(loc)

[49.23206, 49.276774, 49.500151, 49.841375, 51.08277, 49.23206]
[-123.032935, -123.12523, -117.283269, -119.487573, -114.154081, -123.032935]
[(49.23206, -123.032935), (49.276774, -123.12523), (49.500151, -117.283269), (49.841375, -119.487573), (51.08277, -114.154081), (49.23206, -123.032935)]


In [157]:
#Google API

API_KEY=''

def urlparserwaypoint(coordinates):
    n = len(coordinates)
    txt1 = []
    
    # loop all except the first and last
    for coordinate in coordinates[1:-1]:
        txt1.append(','.join(list(map(str, coordinate))))
    txt = 'via:' + '|via:'.join(txt1)
    return urllib.parse.quote(txt)


origin = urllib.parse.quote(','.join(list(map(str, loc[0]))))
waypoints = urlparserwaypoint(loc)

print(waypoints)
url = ("https://maps.googleapis.com/maps/api/directions/json"
       + "?destination={}"
       + "&origin={}"
       + "&waypoints={}"
       + "&key={}").format(origin, origin, waypoints, API_KEY)
payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


via%3A49.276774%2C-123.12523%7Cvia%3A49.500151%2C-117.283269%7Cvia%3A49.841375%2C-119.487573%7Cvia%3A51.08277%2C-114.154081
{
   "geocoded_waypoints" : [
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJzd5M1pF2hlQRoOGbt7bQHwo",
         "types" : [ "atm", "establishment", "finance", "point_of_interest" ]
      },
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJYZoTsqNzhlQR6eIIwOVx0j4",
         "types" : [ "establishment", "point_of_interest", "store" ]
      },
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJCcOgrrK2fFMRYHivFeBoQi8",
         "types" : [ "premise" ]
      },
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJyRfSUA6LfVMRX7KIBlhxqWY",
         "types" : [ "establishment", "point_of_interest" ]
      },
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJTQJkKxxvcVMRgdQrG6Hl5ag",
         "types" : [ "premise" ]
      },
      {
         "geocoder_status" : "OK",
     

In [162]:
with open('direction.json', 'w') as f:
    json.dump(response.json(), f)

In [172]:
gmap = gmplot.GoogleMapPlotter(49.260942911888534, -123.11405788826303, 13, apikey="")

f = open('direction.json')
results = json.load(f)

pos = results["routes"][0]["legs"][0]["steps"]
startlat=results["routes"][0]["legs"][0]["steps"][0]["start_location"]["lat"]
startlong=results["routes"][0]["legs"][0]["steps"][0]["start_location"]["lng"]

leglat = [startlat]
leglong = [startlong]

#put latitude and longitude
for leg in pos:
    leglat.append(leg["end_location"]["lat"])
    leglong.append(leg["end_location"]["lng"])


# Plot the route taken (straight line):
# gmap.plot(lat, long, color='red', edge_width=5)
gmap.plot(leglat, leglong, color='cyan', edge_width=5)

attractions = zip(*loc)
# Mark the distribution center:
gmap.scatter(
    *attractions,
    color=['red', 'orange', 'yellow', 'green', 'blue', 'purple'],
    s=60,
    ew=2,
    marker=True,
    title=[None, None, None, 'Third', None, None],
    label=['A', 'B', 'C', 'D', 'E', 'F']
)

# Draw the map:
gmap.draw('map.html')